# Using the MaterialSolvated

In the [first tutorial](./simple_fitting.rst), we looked at a straight forward example of a polymer film at the solid/liquid interface. 
To analyse this data, we constructed a model with two layers, one SiO<sub>2</sub> and one of the polymer film, and when the analysis was performed, the scattering length density of the polymer film was allowed to vary to find an optimum value. 
However, it is likely that this scattering length density is in fact a compound value arising from the mixture of the polymer with some D<sub>2</sub>O intercalated. 
Therefore, if, for example, the surface covereage of the polymer was known, it may be possible to determine the scattering length density alone. 
Of course, this could be calculated from the optimal scattering length density for the film, but it is more intuitive to include this in our modelling approach. 
Here, we will show how to use the `MaterialSolvated` [type](../sample/material_library.rst#materialsolvated) to perform this analysis. 

First configure matplotlib to place figures in notebook and import needed modules

In [ ]:
%matplotlib inline

import numpy as np
import scipp as sc
import EasyReflectometry
import refnx
from EasyReflectometry.data import load
from EasyReflectometry.sample import Layer
from EasyReflectometry.sample import Sample
from EasyReflectometry.sample import Material
from EasyReflectometry.sample import MaterialSolvated
from EasyReflectometry.sample import Multilayer
from EasyReflectometry.experiment.model import Model
from EasyReflectometry.calculators import CalculatorFactory
from EasyReflectometry.fitting import Fitter
from EasyReflectometry.plot import plot

Showing the version of specific softare for reproducibility

In [ ]:
print(f'numpy: {np.__version__}')
print(f'scipp: {sc.__version__}')
print(f'EasyReflectometry: {EasyReflectometry.__version__}')
print(f'refnx: {refnx.__version__}')

For information about the data being read in and the details of the model see [the previous tutorial](./simple_fitting.rst). 
We will gloss over these details here. 

In [ ]:
data = load('_static/example.ort')

## Constructing the model

Previously the model consisted of four materials, we will construct those again here. 

In [ ]:
si = Material.from_pars(2.07, 0, 'Si')
sio2 = Material.from_pars(3.47, 0, 'SiO2')
film = Material.from_pars(2.0, 0, 'Film')
d2o = Material.from_pars(6.36, 0, 'D2O')

However, now we will construct a component object (for type `MaterialSolvated`), based on the knowledge that there is a 75 % surface coverage of the silicon block by the polymer film. 
Note that this object takes the two materials and then the fractional amount of the second material in the first.

In [ ]:
solvated_film = MaterialSolvated.from_pars(film, d2o, 0.25, name='Solvated Film')

So for the `solvated_film` object, the scattering length density is calculated as,

$$ \rho_{\mathrm{solv film}} = (1-\phi)\rho_{\mathrm{film}} + \phi\rho_{\mathrm{D}_2\mathrm{O}}, $$

where the scattering length densities are given with $\rho$ and the solvation fraction with $\phi$. 
This means that when we investigate the `solvated_film` object, the scattering length density will be 3.09e-6 Å<sup>-2</sup>

In [ ]:
solvated_film

Now, we can construct our layers, sample and model. 

In [ ]:
si_layer = Layer.from_pars(si, 0, 0, 'Si layer')
sio2_layer = Layer.from_pars(sio2, 30, 3, 'SiO2 layer')
solvated_film_layer = Layer.from_pars(solvated_film, 250, 3, 'Film Layer')
superphase = Layer.from_pars(d2o, 0, 3, 'D2O Subphase')

subphase = Multilayer.from_pars([si_layer, sio2_layer], name='Si/SiO2 Superphase')
sample = Sample.from_pars(subphase, solvated_film_layer, superphase, name='Film Structure')
model = Model.from_pars(sample, 1, 1e-6, 0.02, 'Film Model')

## Setting varying parameters

Previously, the scattering length density of the `film_layer` was allowed to vary (in addition to other parameters). 
This time, the scattering length density will for the `film` be varied, leading to a change in the `solvated_film_layer` scattering length density. 
We show this in the four code cells below. 

In [ ]:
film.sld.value = 2.0

In [ ]:
print(solvated_film_layer.material.sld)

In [ ]:
film.sld.value = 2.5

In [ ]:
print(solvated_film_layer.material.sld)

The parameter bounds are then set as follows. 

In [ ]:
# Thicknesses
sio2_layer.thickness.bounds = (15, 50)
solvated_film_layer.thickness.bounds = (200, 300)
# Roughnesses
sio2_layer.roughness.bounds = (1, 15)
solvated_film_layer.roughness.bounds = (1, 15)
superphase.roughness.bounds = (1, 15)
# Scattering length density
film.sld.bounds = (0.1, 3)
# Background
model.background.bounds = (1e-8, 1e-5)
# Scale
model.scale.bounds = (0.5, 1.5)

## Perform the fitting

Having constructed the model and set the relevant varying parameters, we run the analysis (using the default [refnx engine](https://refnx.readthedocs.io/)). 

In [ ]:
interface = CalculatorFactory()
model.interface = interface
fitter = Fitter(model)
analysed = fitter.fit(data)

The model fits well to the data.

In [ ]:
plot(analysed)

We can probe the `MaterialSolvated` object to investigate the scattering length density of the polymer film alone.

In [ ]:
solvated_film

The measured scattering length density (SLD) of the layer is therefore composed of 75% of the polymer film layer (SLD of 1.026E-6 Å<sup>-2</sup>) and 25% (solvation) of D2O (SLD of 6.36E-6  Å<sup>-2</sup>) making (0.75 * 1.026 + 0.25 * 6.36)E-6 = 2.36E-6 Å<sup>-2</sup>.  This is the same as the result from the [previous tutorial](./simple_fitting.rst).